In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-06 04:15:46.957465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 04:15:47.063886: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-06 04:15:47.609783: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Generate the Train/Dev/Test

In [3]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)
loader.sample(0.1)
loader.splitData()

source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

#data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['ElevenLabs'])a

1306
783 1044


In [4]:
data_df.head()

,type,path,label,multiclass_label
0,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [5]:
train_count = data_df[data_df['type'] == 'train'].shape[0]
dev_count = data_df[data_df['type'] == 'dev'].shape[0]
test_count = data_df[data_df['type'] == 'test'].shape[0]

print(f'# of Train instances: {train_count}')
print(f'# of Dev instances: {dev_count}')
print(f'# of Test instances: {test_count}')

# of Train instances: 1566
# of Dev instances: 522
# of Test instances: 524


## Generate the Features

In [6]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
feature_df, feature_cols = embedding_manager.generateFeatureDf()

[NeMo I 2023-04-06 04:15:52 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-04-06 04:15:52 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-04-06 04:15:52 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-04-06 04:15:52 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-04-06 04:15:52 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-04-06 04:15:52 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-04-06 04:15:52 features:267] PADDING: 16
[NeMo I 2023-04-06 04:15:53 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


[NeMo W 2023-04-06 04:15:53 nemo_logging:349] /home/ubuntu/miniconda3/envs/nemo/lib/python3.8/site-packages/nemo/collections/asr/models/label_models.py:437: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525491394/work/torch/csrc/utils/tensor_new.cpp:230.)
      torch.tensor([audio], device=device),
    


In [7]:
#cadence_manager = CadenceModelManager(data_df)
#feature_df, feature_cols =  cadence_manager.run_cadence_feature_extraction_pipeline()

In [8]:
feature_df.head()

,type,path,label,multiclass_label,0,1,2,3,4,5,...,182,183,184,185,186,187,188,189,190,191
0,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.016714,0.005400,-0.009738,0.021128,-0.014102,0.031398,...,0.008778,0.000308,-1.658708e-02,-0.013447,0.006099,0.032921,-0.017553,-0.003184,0.010100,-0.002100
1,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.010898,0.006426,0.012742,0.016855,-0.018741,-0.001058,...,-0.012862,0.009545,-7.136932e-07,-0.011805,0.021345,0.012488,0.013538,0.015600,0.008326,0.003499
2,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.013994,-0.003443,-0.007238,0.008705,-0.013361,0.019492,...,-0.001584,0.002109,-9.996667e-03,-0.025861,0.011085,0.009016,0.015038,0.004588,0.005513,-0.014584
3,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.008663,-0.002662,-0.004435,0.030348,-0.015254,0.022459,...,-0.017928,0.003475,-5.310708e-03,-0.008899,-0.001103,0.022448,0.010539,0.017438,0.007632,0.004290
4,train,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.003007,0.002043,-0.002273,0.035606,-0.019608,0.022570,...,-0.001105,-0.018791,6.015320e-03,-0.020697,0.011641,0.017223,0.002605,0.003883,0.008884,-0.005917


## Predict and Evaluate

In [9]:
model_manager = ModelManager('svm', feature_df, feature_cols, merge_train_dev=True)

In [10]:
acc, logloss = model_manager.trainPredict(label_col='label')

In [11]:
#model_manager.plotRocCurve()

In [12]:
#model_manager.plotProbaDistribution()

In [13]:
print(acc, logloss)

1.0 None
